![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Colab Setup

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/finance-nlp/16.0.earning_calls_preprocessing.ipynb)

In [ ]:
!pip install johnsnowlabs

from johnsnowlabs import nlp,finance

nlp.install(force_browser=True)

In [ ]:

from johnsnowlabs import nlp,finance

params = {"spark.driver.memory":"32G",
"spark.kryoserializer.buffer.max":"2000M",
"spark.driver.maxResultSize":"20000M"}


spark = nlp.start(spark_conf=params)

👌 Launched cpu optimized session with with: 🚀Spark-NLP==5.0.1, 💊Spark-Healthcare==5.0.1, running on ⚡ PySpark==3.1.2


In [ ]:
!unzip /content/2023-07-28-06-50-25_Earning_Call_Transcripts.zip -d data/

Archive:  /content/2023-07-28-06-50-25_Earning_Call_Transcripts.zip
  inflating: data/2023-07-28-06-50-25_Earning_Call_Transcripts/result.json  


In [ ]:
import json

exported_json = "/content/data/2023-07-28-06-50-25_Earning_Call_Transcripts/result.json"

with open(exported_json, "r", encoding="utf8") as f:
    json_content = json.load(f)

json_content[0]

In [ ]:
USERS = ["sorina", "milind"]

completions_filtered = []

for content in json_content:
    completions = [x for x in content['completions'] if x['created_username'] in USERS]
    if len(completions)==0:
        continue
    #last_completion = content['completions'][-1]
    #completions_filtered.append(last_completion['id'])

    completions_filtered.extend(x['id'] for x in completions)

len(completions_filtered)

156

In [ ]:
text_tuples = []
text = []
for content in json_content:
    for completion in content['completions']:
        if completion['id'] in completions_filtered:
            text.append(content['data']['text'])
            text_tuples.append((completion['id'], content['data']['text']))

In [ ]:
import pandas as pd
text_df = pd.DataFrame(text_tuples, columns=['text_id', 'text'])
text_df.head()

,text_id,text
0,138001,\n\nImage source: The Motley Fool.\n\nAon (AON...
1,118001,\n\nImage source: The Motley Fool.\n\nBioCryst...
2,115001,\n\nImage source: The Motley Fool.\n\nPhillips...
3,117001,\n\nImage source: The Motley Fool.\n\nPfizer (...
4,112001,\n\nImage source: The Motley Fool.\n\nYum Chin...


In [ ]:
df_temp = text_df.head(5)

In [ ]:
df_temp.to_csv('ec_head.csv',index=False)

In [ ]:
len(text)

156

In [ ]:
text[0]

"\n\nImage source: The Motley Fool.\n\nAon\xa0(AON 1.07%)Q3\xa02022 Earnings CallOct 28, 2022, 8:30 a.m. ETContents:  Prepared Remarks Questions and Answers Call Participants  Prepared Remarks:  OperatorGood morning and thank you for holding. Welcome to the Aon plc's third quarter 2022 conference call. [Operator instructions] I would also like to remind all parties of this call is being recorded. If anyone has an objection, you may disconnect your line at this time. It is important to note that some of the comments in today's call may constitute certain statements that are forward-looking in nature as defined by the Private Securities Reform Act of 1995. Such statements are subject to certain risks and uncertainties that could cause actual results to differ materially from historical results or those anticipated. Information concerning risk factors that could cause such differences are described in the press release covering our third quarter 2022 results as well as having been posted 

In [ ]:
paragraphs = []

for i in text:
  paras = i.split('\n\n')
  temp = []
  temp_para = ''
  for j in range(len(paras)):
    text_split = paras[j].split(' ')
    if len(text_split) < 100:
      temp_para += paras[j]
      temp_para += ' '
    else:
      text = temp_para + paras[j]
      temp_para = ''
      paragraphs.append(text)






In [ ]:
#Taking just a part of the data
test_paras = paragraphs

In [ ]:
final = []


In [ ]:
for i in range(len(test_paras)):
  split = test_paras[i].split(' ')
  temp = ''
  if len(split) > 512:
    test_paras[i] = test_paras[i][:512]

  final.append(test_paras[i])



In [ ]:
len(final)

3902

In [ ]:
import pandas as pd

df = pd.DataFrame(final,columns=['text'])

In [ ]:
df

,text
0,Image source: The Motley Fool. Aon (AON 1.07%...
1,Greg Case -- Chief Executive Officer Good mo...
2,"And third, they value expert insight and are..."
3,These results are consistent with our full y...
4,Christa Davies -- Chief Financial Officer and...
...,...
3897,Richard Baldry -- ROTH Capital Partners -- An...
3898,DJ Hynes -- Canaccord Genuity -- Analyst Yeah...
3899,DJ Hynes -- Canaccord Genuity -- Analyst Yes....
3900,Ryan MacWilliams -- Barclays -- Analyst Appre...


In [ ]:
df.to_csv('full_subpoenas.csv',index=False)

In [ ]:
df.shape

(3902, 1)

In [ ]:
df_new = df.head(100)

In [ ]:
df_new.to_csv('100_subpoenas.csv',index=False)

In [ ]:
df_new.shape

(100, 1)

In [ ]:
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/finance-nlp/data/finance_subpoenas.csv

--2023-08-07 06:03:15--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/finance-nlp/data/finance_subpoenas.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 125359 (122K) [text/plain]
Saving to: ‘finance_subpoenas.csv’

finance_subpoenas.c 100%[===================>] 122.42K  --.-KB/s    in 0.03s   

2023-08-07 06:03:16 (4.69 MB/s) - ‘finance_subpoenas.csv’ saved [125359/125359]

